In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 15


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.378983,-76.541061
2,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
3,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
4,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-10-01T22:58:24-05:00 -> 1696219104000, Until: 2023-10-06T22:58:24-05:00 -> 1696651104000
Group by Time: 30s


In [4]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp', GROUP_BY_TIME)
df_temp_raw.head()
#df_temp_raw.shape

,TANGARA_2BBA,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,,,
2023-10-01 22:58:00-05:00,-147.2,NaN,NaN,27.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.09
2023-10-01 22:58:30-05:00,-147.2,NaN,28.94,27.49,30.04,30.08,28.22,30.64,28.44,28.10,26.69,28.2,28.2,NaN,29.11
2023-10-01 22:59:00-05:00,-147.2,NaN,28.92,27.51,30.01,30.11,28.22,30.67,28.47,28.12,26.67,28.2,28.2,NaN,29.10
2023-10-01 22:59:30-05:00,-147.2,NaN,NaN,27.49,29.98,30.11,28.22,30.67,28.42,28.10,26.67,28.1,28.2,NaN,29.10
2023-10-01 23:00:00-05:00,-147.2,NaN,28.96,27.49,29.99,30.11,28.20,30.69,28.45,28.10,26.64,28.1,28.1,NaN,29.09


## Descriptive Statistics

In [5]:
# Describe Data
df_temp_raw.describe()

,TANGARA_2BBA,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,5.960000e+03,1379.0,5827.000000,5781.000000,2798.000000,5546.000000,5534.000000,5796.000000,5535.000000,5802.000000,5625.000000,5713.000000,5911.000000,4408.000000,5919.000000
mean,-1.472000e+02,0.0,30.084229,30.021604,31.832144,29.329762,28.366482,31.935122,30.898195,31.352416,28.449321,30.401033,28.588936,31.265824,30.525852
std,2.842409e-14,0.0,2.355224,3.959368,3.541345,2.118132,2.179128,2.729657,3.768716,4.326969,2.446738,3.822810,3.153052,2.585443,3.331139
min,-1.472000e+02,0.0,25.460000,23.850000,27.520000,25.670000,24.820000,28.380000,25.470000,26.390000,24.550000,24.700000,23.800000,26.590000,25.670000
25%,-1.472000e+02,0.0,28.340000,26.680000,28.430000,27.860000,26.880000,29.550000,27.310000,27.450000,26.470000,27.100000,25.800000,29.220000,27.430000
50%,-1.472000e+02,0.0,29.970000,29.350000,30.810000,29.520000,28.230000,30.940000,30.660000,29.970000,27.730000,30.000000,28.100000,31.630000,30.570000
75%,-1.472000e+02,0.0,32.060000,33.260000,34.772500,30.980000,29.460000,34.280000,34.340000,35.150000,30.910000,33.700000,31.300000,33.430000,33.370000
max,-1.472000e+02,0.0,34.980000,37.700000,38.630000,33.210000,36.920000,37.840000,38.060000,41.320000,33.210000,40.400000,35.100000,35.810000,37.830000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_2BBA, Data: 41%, Missing: 59%, To be checked
Tangara Sensor: TANGARA_25CE, Data: 10%, Missing: 90%, To be checked
Tangara Sensor: TANGARA_260A, Data: 40%, Missing: 60%, To be checked
Tangara Sensor: TANGARA_4B1A, Data: 40%, Missing: 60%, To be checked
Tangara Sensor: TANGARA_14D6, Data: 19%, Missing: 81%, To be checked
Tangara Sensor: TANGARA_1EE6, Data: 39%, Missing: 61%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 38%, Missing: 62%, To be checked
Tangara Sensor: TANGARA_2E9A, Data: 40%, Missing: 60%, To be checked
Tangara Sensor: TANGARA_2FF6, Data: 38%, Missing: 62%, To be checked
Tangara Sensor: TANGARA_307A, Data: 40%, Missing: 60%, To be checked
Tangara Sensor: TANGARA_1712, Data: 39%, Missing: 61%, To be checked
Tangara Sensor: TANGARA_48C6, Data: 40%, Missing: 60%, To be checked
Tangara Sensor: TANGARA_532E, Data: 41%, Missing: 59%, To be checked
Tangara Sensor: TANGARA_F1AE, Data: 31%, Missing: 69%, To be checked
Tangara Sensor: TAN

## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")